In [1]:
import json
import glob
import re
import pandas as pd
import os

In [2]:
root="/home/jovyan/work"
analysis_results_path = root + "/analysis/results/"

In [26]:
projects = [
    "JacksonXml", "Time", "Collections", "Compress", "Csv", "JacksonCore", "JacksonDatabind", "Gson", "Jsoup",
    "Lang", "Math", "Closure"
]
regressions = []
for project in projects:
    # FOR EACH BUG
    for bug_path in glob.glob("{root}/results/{project}/Bug_*/".format(root=root, project=project)):
        bug_id = re.search(r"Bug_(\d+)", bug_path).group(1)
        result_path = root+"/analysis/results/{project}/Bug_{bug_id}/bug_result.json".format(project=project, bug_id=bug_id)
        # analysis/results/JacksonXml/Bug_1/bug_result.json
        with open(result_path) as f:
            result = json.load(f)
            if result['category'] == "A regression is detected": #result['sub_category'] == "Unique candidates": 
                regressions.append(result)

In [28]:
pd.DataFrame(regressions)[['id', 'sub_category', 'BIC_candidates']]

,id,sub_category,BIC_candidates
0,JacksonXml_Bug_3,Multiple candidates,"[[163, eb9ac89e35d7ca73fc054b03c29d7bcc4be7ffc..."
1,Time_Bug_1,Unique candidates,"[[1, 8612f9e5b88c1bea933ef9ab1e431f5db3006b48]]"
2,Compress_Bug_28,Unique candidates,"[[273, a671a703fe86b69adb8c61495a915065c186362d]]"
3,Compress_Bug_45,Unique candidates,"[[1, 137aa57f9291b5f390de740266042587cfede7ce]]"
4,JacksonCore_Bug_11,Unique candidates,"[[162, 30a2670b336c3a4e4dee57460577b4bcc4b9e9b6]]"
...,...,...,...
57,Closure_Bug_131,Multiple candidates,"[[1480, 9360326ac90197b8740b23146ffd84f7e18861..."
58,Closure_Bug_113,Unique candidates,"[[8, 87071ddf324c3885361f0375b84e15c7cd62283d]]"
59,Closure_Bug_120,Unique candidates,"[[3, 000ea0f8d6684bb9ff84a483d4ff44952b29d3f7]]"
60,Closure_Bug_115,Unique candidates,"[[5, b80dfd3b0e5e4b490b7afb469320d18bac4520b4]]"


In [16]:
def simplyfy_candidate(reg):
    reg['BIC'] = reg['BIC_candidates'][0][1]
    return reg
regressions = list(map(lambda r: simplyfy_candidate(r), regressions))
pd.DataFrame(regressions)[['id', 'BIC']]

,id,BIC
0,Time_Bug_1,8612f9e5b88c1bea933ef9ab1e431f5db3006b48
1,Compress_Bug_28,a671a703fe86b69adb8c61495a915065c186362d
2,Compress_Bug_45,137aa57f9291b5f390de740266042587cfede7ce
3,JacksonCore_Bug_11,30a2670b336c3a4e4dee57460577b4bcc4b9e9b6
4,JacksonCore_Bug_10,88391ccd5f494490d183306368da42b79b36cd26
5,JacksonCore_Bug_21,fd32772bb402f6cc2ffea5f58c162f86a8aa81e3
6,JacksonDatabind_Bug_24,c89a98af57bda92ce5d1884381f008200b174d31
7,JacksonDatabind_Bug_35,a6443b2467542314065ce4545bfb52d5df2a76ed
8,JacksonDatabind_Bug_41,83e514792c9cadaeaffceed47fd1c5ad4715fda1
9,JacksonDatabind_Bug_87,39785388b05ed0ac447860d96015fce359f1ada6


In [5]:
for regression in regressions:
    # print(regression['project'], regression['bug'])
    
    # SZZ UNLEASHED
    szz_unleashed_result_path = root+"/results/szz/SZZUnleashed/{id}_SZZUnleashed/results/fix_and_introducers_pairs.json".format(id=regression['id'])
    if os.path.isfile(szz_unleashed_result_path):
        with open(szz_unleashed_result_path) as f:
            szz_unleashed_result = json.load(f)
            szz_unleashed_result = list(map(lambda c: c[1],szz_unleashed_result))
            regression['SZZ_UNLEASHED_CANDIDATES'] = len(szz_unleashed_result) 
            regression['SZZ_UNLEASHED'] = regression['BIC'] in szz_unleashed_result 
    
    # OPEN SZZ
    
    openszz_result_path = root+"/results/szz/OpenSZZ/{id}_OpenSZZ/suspects.json".format(id=regression['id'])
    if os.path.isfile(openszz_result_path):
        with open(openszz_result_path) as f:
            openszz_result = json.load(f)
            regression['OPENSZZ_LEN_CANDIDATES'] = len(openszz_result) 
            regression['OPENSZZ'] = regression['BIC'] in openszz_result 
            
    
    # PYSZZ
    for alg in ['ag', 'l', 'r', 'ma', 'ra']:
        pyszz_result_path = root+"/results/szz/PySZZ_{alg}/{id}_PySZZ_{alg}/result.json".format(id=regression['id'],alg=alg)
        if os.path.isfile(pyszz_result_path):
            with open(pyszz_result_path) as f:
                pyszz_result = json.load(f)[0]['inducing_commit_hash']
                regression['PYSZZ_{alg}_LEN_CANDIDATES'.format(alg=alg)] = len(pyszz_result) 
                regression['PYSZZ_{alg}'.format(alg=alg)] = regression['BIC'] in pyszz_result

In [6]:
pd.DataFrame(regressions)[
    ['id', 'OPENSZZ', 'SZZ_UNLEASHED', 'PYSZZ_ag', 'PYSZZ_l', 'PYSZZ_r', 'PYSZZ_ma', 'PYSZZ_ra']
]

,id,OPENSZZ,SZZ_UNLEASHED,PYSZZ_ag,PYSZZ_l,PYSZZ_r,PYSZZ_ma,PYSZZ_ra
0,Time_Bug_1,False,False,True,False,False,True,True
1,Compress_Bug_28,True,False,True,True,True,True,True
2,Compress_Bug_45,False,False,False,False,False,False,False
3,JacksonCore_Bug_11,False,False,False,False,False,False,False
4,JacksonCore_Bug_10,False,False,False,False,False,False,False
5,JacksonCore_Bug_21,True,False,False,True,True,True,True
6,JacksonDatabind_Bug_24,False,False,False,False,True,True,True
7,JacksonDatabind_Bug_35,False,False,False,False,False,False,False
8,JacksonDatabind_Bug_41,False,False,True,False,True,True,True
9,JacksonDatabind_Bug_87,True,False,False,False,True,True,False


In [7]:
pd.DataFrame(regressions).to_csv("regressions.csv")